In [17]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [18]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [19]:
# On Simple English wiki, POS tagged

sf = open(datapath+'/Corpora/wiki/simple_20200601/Tagged/simple_20200601_tagged.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

April|NNP is|VBZ the|DT fourth|JJ month|NN of|IN the|DT year|NN and|CC comes|VBZ between|IN March|NNP and|CC May|NNP 

It|PRP is|VBZ one|CD of|IN four|CD months|NNS to|TO have|VB 30|CD days|NNS 

April|NNP always|RB begins|VBZ on|IN the|DT same|JJ day|NN of|IN week|NN as|IN July|NNP and|CC additionally|RB January|NNP in|IN leap|NNP years|NNS 

April|NNP always|RB ends|VBZ on|IN the|DT same|JJ day|NN of|IN the|DT week|NN as|IN December|NNP 

April|NNP 's|POS flowers|NNS are|VBP the|DT Sweet|NNP Pea|NN and|CC Daisy|NNP 



In [20]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

from nltk.corpus.reader.util import read_line_block

In [21]:
simp_t = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/Tagged','simple_20200601_tagged.txt',
                            word_tokenizer = WhitespaceTokenizer(),
                            para_block_reader=read_line_block
                            )

In [22]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [23]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [24]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [25]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 '4|CD',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'born|VBN',
 'by|IN',
 'can|MD',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'people|NNS',
 's|POS',
 'that|IN',
 'that|WDT',
 'they|PRP',
 'the|DT',
 'to|IN',
 'to|TO',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'with|IN',
 '|HYPH',
 '–|SYM'}

In [26]:
min_freq = 20
eval_count = 150000

In [27]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Grădina|NNP, Zoologică|NNP)",20,-606.668931,2,1
1,"(Karlovy|NNP, Vary|NNP)",20,-606.668931,2,1
2,"(Ving|NNP, Rhames|NNP)",20,-606.668931,2,1
3,"(waystations|NNS, shuku|NN)",20,-608.076717,2,1
4,"(Aer|NNP, Lingus|NNP)",20,-609.419001,2,1
...,...,...,...,...,...
149995,"(Cena|NNP, and|CC)",56,-2732.579707,2,5
149996,"(and|CC, keeping|VBG)",56,-2732.579707,2,1
149997,"(province|NN, has|VBZ, a|DT)",40,-2732.586816,3,6
149998,"(speak|VBP, a|DT)",56,-2732.604570,2,2


In [28]:
batch_count = max(ngram_eval.batch)

In [29]:
ngram_eval.batch.value_counts()

 1     47772
 2     33213
 3     25419
 4     15895
 5     10517
 6      6533
 7      3782
 8      2398
 9      1434
 10     1073
-2       732
 11      317
-1       282
 14      239
 12      227
 13       96
 15       71
Name: batch, dtype: int64

In [30]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [31]:
for s in Sent_Seq(simp_t, remchars = "[`¬@^\"]"):
    print(s)
    break

['April|NNP', 'is|VBZ', 'the|DT', 'fourth|JJ', 'month|NN', 'of|IN', 'the|DT', 'year|NN', 'and|CC', 'comes|VBZ', 'between|IN', 'March|NNP', 'and|CC', 'May|NNP']


In [32]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Build model
    print('Building word2vec model')
    sents_mwe = Sent_Seq(simp_t, tokenizer = batch_token_mwe, remchars = "[`¬@^\"]")
    
    print(' Training model')
    batch_model = Word2Vec(sents_mwe,
                             min_count = 20,  # 20 matches R&E on EN Wiki
                             size = 400,
                             workers = 8,
                             window = 5,
                             sg = 0,         # CBOW
                             sample = 10e-5, # Subsampling
                             negative = 10
                            )

    # Save model
    print(' Saving model')
    batch_model.save(datapath+'/Models/1 w2v/Tagged/simp_tagged_batch{}.model'.format(bb+1))
    


Processing batch 1 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 2 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 3 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 4 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 5 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 6 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 7 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 8 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 9 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 10 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 11 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 12 of 15
Building word2vec model
 Training model
 Saving model
Processing batch 13 of 15
Building word2vec model